In [1]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np
from scipy.stats import poisson

In [51]:
def readPreferenceGraph(filePath) :
    with open("./preferenceGraph.json") as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], comment=edge["comment"])
    
    return G

In [54]:
def interactionGraph(preferenceGraph, personalityThreshold, maxInteractions, maxTimestamp) :
   def analyzePreferenceGraph(timestamp) :
      def calculateProbability(weight) :
         if re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
               return 1
         elif re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
            return 0.7
         elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
            return 0.3
         else :
            return 0
      
      #dictionary {edge : probability of existing in the Preference Graph}
      edgesProbabilities = {}

      #examine each eage in Preference Graph
      #calculate probability
      for u, v in preferenceGraph.edges() :
         if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
            if (v,u) in edgesProbabilities :
                #(v,u) is already checked, so neglect (u,v)
               continue 
            else :
               #(u,v), (v,u) pair has not been checked
               edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["comment"]) + calculateProbability(preferenceGraph[v][u]["comment"]))
         else :
            edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["comment"])

      #create edge in Interaction Graph with respect to the existing edge probability
      for edge, prob in edgesProbabilities.items() :
         if np.random.choice([0,1], p=[1-prob, prob]) == 1 :
            interactionGraph.add_interaction(edge[0], edge[1], timestamp)
      
      return edgesProbabilities
      
      
   def analyzeStudentsPreferences(timestamp) :
      def checkProbSum(arr) :
         diff = 1-sum(arr)
         if diff > 0 :
            arr[-1] += abs(diff)
         elif diff < 0 :
            arr[-1] -= abs(diff)
         else :
            None
         return arr
      #get a snapshot at the timestamp where students' preferences were added      
      #calculate degree of nodes
      snapshotDegrees = interactionGraph.degree(t=timestamp)

      #detect the maximum degree
      maxDegree = max(snapshotDegrees.values())

      #find the ratio of a node over the maximum
      for node, degree in snapshotDegrees.items() :
         snapshotDegrees[node] = degree/maxDegree
      
      #split students in low and high profile based on a threshold
      lowProfileStudents = set()
      highProfileStudents = set()
      for node, degree in snapshotDegrees.items() :
         if degree < personalityThreshold :
            lowProfileStudents.add(node)
         else :
            highProfileStudents.add(node)
      
      #distribution of each students' category
      lowProfileDistribution = poisson.pmf(range(5), mu=0.8)
      highProfileDistribution = poisson.pmf(range(5), mu=1.5)


      return lowProfileStudents, highProfileStudents, checkProbSum(lowProfileDistribution), checkProbSum(highProfileDistribution)
  

   def createRandomEdgesAtTimestamp(timestamp) :
      nodes = preferenceGraph.nodes()

      #function to get number of interactions from 0,...,timestamp-1 of node
      def previousDegree(node) :
         totalDegree = 0
         for time in range(timestamp) :
            totalDegree += interactionGraph.degree(node, t=time)
         
         return totalDegree


      for node in nodes :
         #for a specific node

         #find distribution with respect to the category of the node
         #find number of interactions to add as long as it's less than the total possible else do nothing
         profileDistribution = lowProfileDistribution if node in lowProfileStudents else highProfileDistribution
         numberOfInteractions = np.random.choice(range(len(profileDistribution)), p=profileDistribution)

         possibleInteractions = maxInteractions - previousDegree(node)
         if possibleInteractions > 0 :
            numberOfInteractions = min(numberOfInteractions, possibleInteractions)
         else :
            continue

         #create a list of the nodes that it could possibly interact with
         #the node itself and the nodes that avoided it in Preference Graph are excluded
         newInteractions = []
         for other in nodes :
            if ((node, other) in edgesProbabilities and edgesProbabilities[(node, other)] == 0) \
               or ((other, node) in edgesProbabilities and edgesProbabilities[(other, node)] == 0) \
               or other == node :
               continue
            newInteractions.append(other) 
         
               
 
         if numberOfInteractions == 0 :
            continue
         
         #all nodes to interaction have equal probability to interact with
         for interaction in np.random.choice(newInteractions, size=numberOfInteractions, replace=False) :
            interactionGraph.add_interaction(node, interaction, t=timestamp)

              
   #create an empty Interaction Graph
   interactionGraph = dn.DynGraph()
   
   #interactions at timetsamp t = 0
   #interactions based on a scale-free network model
   G = nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3)
   interactionGraph.add_interactions_from(G.edges(), t=0)
   
   #interactions at timestamp t = 1
   #interactions based on Preference Graph (students' preferences for their classmates)
   edgesProbabilities = analyzePreferenceGraph(timestamp=1)

   #analyze students' preferences
   #Based on the personalityThreshold, split students in 2-categories: low profile and high profile
   lowProfileStudents, highProfileStudents, lowProfileDistribution, highProfileDistribution = analyzeStudentsPreferences(timestamp=1)
   
   #interactions at timestamp t >= 2
   for time in range(2, maxTimestamp + 1) :
      createRandomEdgesAtTimestamp(timestamp=time)
   
   
   return interactionGraph

In [55]:
path = "./preferenceGraph.json"
G = interactionGraph(preferenceGraph=readPreferenceGraph(path), personalityThreshold=0.4, maxInteractions=15, maxTimestamp=5)